In [ ]:
import pandas as pd
from bokeh.plotting import figure, ColumnDataSource, show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models import HoverTool, CategoricalColorMapper
from bokeh.palettes import Set1_9 as PALETTE

output_notebook()

In [ ]:
df = pd.read_csv('results.csv', sep=' ')
df = df.dropna()
df.head()

In [ ]:
grid = []
for subspace_source in df['subspace_pairs'].unique():
    row = []
    for debiasing in df['debiasing'].unique():
        if debiasing == 'none':
            continue
        fig = figure(
            width=400, height=300,
            x_range=['none', debiasing],
            y_range=[0, 0.0045],
            title=f'Debiasing {subspace_source} with {debiasing}',
        )
        renderers = []
        plot_df = df[
            (df['subspace_pairs'] == subspace_source)
            & (df['debiasing'].isin(['none', debiasing]))
        ].copy()
        #plot_df['line_color'] = df['subspace_algo'].apply(lambda algo: PALETTE[0] if algo == 'mean' else PALETTE[1])
        uniq_values = plot_df[['subspace_seed', 'subspace_algo', 'bias_words']].drop_duplicates()
        for subspace_seed, subspace_algo, bias_words in uniq_values.itertuples(index=False):
            renderer = fig.line(
                x='debiasing',
                y='bias',
                line_color=(PALETTE[0] if subspace_algo == 'mean' else PALETTE[1]),
                source=ColumnDataSource(plot_df[
                    (plot_df['subspace_seed'] == subspace_seed)
                    & (plot_df['subspace_algo'] == subspace_algo)
                    & (plot_df['bias_words'] == bias_words)
                ]),
            )
            renderers.append(renderer)
        fig.add_tools(HoverTool(renderers=renderers, tooltips=[
            ('subspace_seed', '@subspace_seed'),
            ('subspace_algo', '@subspace_algo'),
            ('bias_words', '@bias_words'),
            ('bias', '@bias'),
        ]))
        row.append(fig)
    grid.append(row)
show(gridplot(grid))